In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

  Activating environment at `D:\Documents\git_workspace\flyhopper\scripts\Julia\Project.toml`
Precompiling project...
  ✓ Mux
  ✓ WebIO
  ✓ CSSUtil
  ✓ JSExpr
  ✓ Knockout
  ✓ Blink
  ✓ InteractBase
  ✓ MeshCat
  ✓ MeshCatMechanisms
  9 dependencies successfully precompiled in 92 seconds (159 already precompiled)


In [2]:
using RigidBodyDynamics
using LinearAlgebra
using MeshCatMechanisms
using MeshCat
using StaticArrays

WebIO._IJuliaInit()

In [3]:
l0 = 0.15
l1 = 0.3

l2 = 0.3
l3 = 0.15
l4 = 0.15
l34 = l3+l4

m0 = 1.0
m1 = 1.0
m2 = 1.0
m3 = 1.0

g = 9.81

# I1 = m1*(l1^2)
# I2 = m2*(l2^2)

9.81

In [4]:
world = RigidBody{Float64}("world")
doublependulum = Mechanism(world; gravity = SVector(0, 0, g))

Spanning tree:
Vertex: world (root)
No non-tree joints.

In [5]:
curdir = pwd()
urdfpath = joinpath(curdir, "../../res/flyhopper_parallel/urdf/flyhopper_parallel.urdf")
doublependulum = parse_urdf(urdfpath, floating=false)
state = MechanismState(doublependulum)

MechanismState{Float64, Float64, Float64, …}(…)

In [6]:
function f_1(x)
    # double-double pendulum dynamics
    q0 = copy(x[1])
    q0d = copy(x[2])
    q1 = copy(x[3])
    q1d = copy(x[4])
    q2 = copy(x[5])
    q2d = copy(x[6])
    q3 = copy(x[7])
    q3d = copy(x[8])
    
    checkval(q0d, 1e3, "top of dyn")
    # Mass matrix
    M = [ 1.0*l0^2*m0 + l0^2*m1 + 2*l0*l1*m1*cos(q1) + l1^2*m1  l0*l1*m1*cos(q1) + l1^2*m1  0  0;
          l0*l1*m1*cos(q1) + l1^2*m1  l1^2*m1  0  0;
          0  0  1.0*l2^2*m2 + l2^2*m3 + 2*l2*l34*m3*cos(q3) + l34^2*m3  l2*l34*m3*cos(q3) + l34^2*m3;
          0  0  l2*l34*m3*cos(q3) + l34^2*m3  l34^2*m3]
    
    # Coriolis term
    C = [ -2*l0*l1*m1*q0d*q1d*sin(q1) - l0*l1*m1*q1d^2*sin(q1);
          l0*l1*m1*q0d^2*sin(q1);
          -2*l2*l34*m3*q2d*q3d*sin(q3) - l2*l34*m3*q3d^2*sin(q3);
          l2*l34*m3*q2d^2*sin(q3)]
    
    # Gravity term
    G = [g*l0*m0*cos(q0) + g*l0*m1*cos(q0) + g*l1*m1*cos(q0 + q1);
         g*l1*m1*cos(q0 + q1);
         g*l2*m2*cos(q2) + g*l2*m3*cos(q2) + g*l34*m3*cos(q2 + q3);
         g*l34*m3*cos(q2 + q3)]

    # Constraint Jacobian
    D = [-l0*sin(q0) - l1*sin(q0 + q1)  -l1*sin(q0 + q1)  l2*sin(q2) + l3*sin(q2 + q3)  l3*sin(q2 + q3);
        l0*cos(q0) + l1*cos(q0 + q1)    l1*cos(q0 + q1)   -l2*cos(q2) - l3*cos(q2 + q3) -l3*cos(q2 + q3)]
    
    d = [q0d*(-l1*q1d*cos(q0 + q1) + q0d*(-l0*cos(q0) - l1*cos(q0 + q1))) + q1d*(-l1*q0d*cos(q0 + q1) -
            l1*q1d*cos(q0 + q1)) + q2d*(l3*q3d*cos(q2 + q3) + q2d*(l2*cos(q2) + l3*cos(q2 + q3))) +
            q3d*(l3*q2d*cos(q2 + q3) + l3*q3d*cos(q2 + q3));
         q0d*(-l1*q1d*sin(q0 + q1) + q0d*(-l0*sin(q0) - l1*sin(q0 + q1))) + q1d*(-l1*q0d*sin(q0 + q1) -
            l1*q1d*sin(q0 + q1)) + q2d*(l3*q3d*sin(q2 + q3) + q2d*(l2*sin(q2) + l3*sin(q2 + q3))) +
            q3d*(l3*q2d*sin(q2 + q3) + l3*q3d*sin(q2 + q3))]
    
    KKT = [M -D';
           D zeros(2,2)]
    RHS = [- G - C;
           d]
    
    sol = KKT\RHS
    qdd = copy(sol[1:4])
    
    λ = copy(sol[5:6])
    # qdd = M\(- G - C)
    q0dd = qdd[1]
    q1dd = qdd[2]
    q2dd = qdd[3]
    q3dd = qdd[4]
    
    ẋ = zeros(8)
    ẋ[1] = q0d # q0 dot
    ẋ[2] = q0dd # q0 double dot
    ẋ[3] = q1d # q1 dot
    ẋ[4] = q1dd # q1 double dot
    ẋ[5] = q2d # q2 dot
    ẋ[6] = q2dd # q2 double dot
    ẋ[7] = q3d # q3 dot
    ẋ[8] = q3dd # q3 double dot
    checkval(q0d, 1e3, "bottom of dyn") 
    return copy(ẋ)
end

f_1 (generic function with 1 method)

In [7]:
function checkval(val, max, name)
    if val > max
        @show val
        error(name)
    end
end
#=
#---#
if isnan(qdd[1])
    @show qdd
    error("A!")
end
#---#
=#

checkval (generic function with 1 method)

In [8]:
function rk4_step(f,xk,h)

    a = [0 0 0 0; 
         0.5 0 0 0; 
         0 0.5 0 0; 
         0 0 1 0]
    b = [1/6, 1/3, 1/3, 1/6]
    
    f1 = f(xk)
    
    checkval(f1[2], 1e3, "f1")
    
    f2 = f(xk .+ h*a[2, 1]*f1)
    
    checkval(f2[2], 1e3, "f2")
    
    f3 = f(xk .+ h*a[3, 1]*f1 .+ h*a[3, 2]*f2)
    
    checkval(f3[2], 1e3, "f3")
    
    f4 = f(xk .+ h*a[4, 1]*f1 .+ h*a[4, 2]*f2 .+ h*a[4, 3]*f3)
    
    checkval(f4[2], 1e3, "f4")
    
    xn = xk .+ h*(b[1]*f1 .+ b[2]*f2 .+ b[3]*f3 .+ b[4]*f4)
    checkval(xn[2], 1e3, "xn")
    return xn
end

rk4_step (generic function with 1 method)

In [9]:
Tf = 10.0
h = 0.001 #20 Hz
N = Int(floor(Tf./h + 1))
thist = h.*Array(0:(N-1));

In [10]:
function simulate!(xtraj, N)
    for k = 1:(N-1)
        checkval(xtraj[2, k], 1e3, "before step")
        xtraj[:,k+1] .= rk4_step(f_1, copy(xtraj[:,k]), h)
        checkval(xtraj[2, k], 1e3, "after step")
    end
end

simulate! (generic function with 1 method)

In [11]:
x0 = [-30*pi/180; 0.0; -120*(pi/180); 0.0; -150*(pi/180); 0.0; 120*(pi/180); 0.0]
xtraj = zeros(8,N)
@show xtraj[:,1] = x0;

xtraj[:, 1] = x0 = [-0.5235987755982988, 0.0, -2.0943951023931953, 0.0, -2.6179938779914944, 0.0, 2.0943951023931953, 0.0]


In [12]:
simulate!(xtraj, N)

val = 1002.9483293203118


LoadError: f2

In [13]:
q0 = copy(-xtraj[1, :] .- 30*(pi/180))
q1 = copy(-xtraj[3, :] .- 120*(pi/180))
q2 = copy(-xtraj[5, :] .- 150*(pi/180))
q3 = copy(-xtraj[7, :] .+ 120*(pi/180))

qs = convert(AbstractArray{Float64}, [q2 q0 q3 q1]) 
ts = convert(AbstractVector{Float64}, thist) # AbstractVector(thist)
q_array = [ qs[i,:] for i in 1:size(qs,1)] 

10001-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.0, 0.0]
 [-1.0788228971048142e-5, 1.34852862139212e-5, 2.427351518496934e-5, -2.427351518496934e-5]
 [-4.3153237479831574e-5, 5.394154684990049e-5, 9.709478432951002e-5, -9.709478432951002e-5]
 [-9.709599042473371e-5, 0.00012136998803302657, 0.0002184659784578713, -0.0002184659784578713]
 [-0.0001726180963474988, 0.00021577262044636392, 0.00038839071679408477, -0.00038839071679408477]
 [-0.00026972180799633705, 0.0003371522600420507, 0.0006068740680387208, -0.0006068740680387208]
 [-0.00038841002311817974, 0.0004855125290382789, 0.0008739225521572358, -0.0008739225521572358]
 [-0.0005286862854765317, 0.0006608578572018242, 0.001189544142679022, -0.001189544142679022]
 [-0.0006905547860949213, 0.0008631934834164579, 0.0015537482695120453, -0.0015537482695120453]
 [-0.0008740203647299438, 0.0010925254575389065, 0.0019665458222695165, -0.0019665458222695165]
 [-0.0010790885115707916, 0.001348860642542804, 0.0024279491541143727, -0.00242

In [14]:
# mvis = MechanismVisualizer(doublependulum, Skeleton(randomize_colors=true, inertias=false));
mvis = MechanismVisualizer(doublependulum, URDFVisuals(urdfpath));

render(mvis)

┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat C:\Users\bboks\.julia\packages\MeshCat\GlCMx\src\visualizer.jl:73


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("meshcat" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("world" => MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("base_link" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x2f, 0x62, 0x61, 0x73, 0x65, 0x5f, 0x6c, 0x69, 0x6e, 0x6b], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_1" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Link 1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6c, 0x64, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 1" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x31], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_2" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Link 2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x6b, 0x20, 0x31, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 2" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x32], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_3" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))))))), "Link 3" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x72, 0x6c, 0x64, 0x2f, 0x4c, 0x69, 0x6e, 0x6b, 0x20, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("after_Joint 3" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x65, 0x72, 0x5f, 0x4a, 0x6f, 0x69, 0x6e, 0x74, 0x20, 0x33], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}("geometry_4" => MeshCat.SceneTrees.SceneNode(UInt8[0x83, 0xa6, 0x6f, 0x62, 0x6a, 0x65, 0x63, 0x74, 0x84, 0xaa  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72, 0x69, 0x78, 0xc7, 0x40  …  0x67, 0x65, 0x6f, 0x6d, 0x65, 0x74, 0x72, 0x79, 0x5f, 0x34], Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()))), "Link 4" => MeshCat.SceneTrees.SceneNode(nothing, UInt8[0x83, 0xa6, 0x6d, 0x61, 0x74, 0x72

In [15]:
# set_configuration!(mvis, [x0[1]-150*(pi/180), x0[3]+120*(pi/180)])
# [q2 q0 q3 q1
set_configuration!(mvis, [-x0[5]-150*(pi/180), -x0[1]-30*(pi/180), -x0[7]+120*(pi/180), -x0[3]-120*(pi/180)])
# set_configuration!(mvis, [0, 0])

In [16]:
# Now we can simply call `simulate`, which will return a tuple consisting of:
# * simulation times (a `Vector` of numbers)
# * joint configuration vectors (a `Vector` of `Vector`s)
# * joint velocity vectors (a `Vector` of `Vector`s)

# MeshCatMechanisms.animate(mvis, ts, qs; realtimerate = 1.);
animation = Animation(mvis, ts, -q_array)
setanimation!(mvis, animation)